
# MAESTRO Transformer Continuation

This notebook trains a compact Transformer model on the [MAESTRO](https://magenta.tensorflow.org/datasets/maestro) piano performance dataset and demonstrates how to continue a piano roll sequence after seeding it with two seconds of music. The workflow focuses on simplicity and reproducibility so that the complete pipeline can run on a single GPU machine.



## 1. Environment

Install the minimal set of packages required for MIDI handling and model training. All packages are available on PyPI. Execute this cell once per environment.


In [5]:

%pip install --quiet pretty_midi tqdm


Note: you may need to restart the kernel to use updated packages.



## 2. Configuration

Set up paths and hyperparameters. By default the notebook expects the MAESTRO dataset to be extracted in a local `maestro/` folder. You can override this location by setting the `MAESTRO_ROOT` environment variable before launching the notebook. Because the full dataset is large (~100 hours of audio), you can limit the number of files processed during experimentation with `MAX_FILES`.


In [1]:
import torch

print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"PyTorch version: {torch.__version__}")
print(f"Device count: {torch.cuda.device_count()}")

CUDA available: True
CUDA version: 12.6
PyTorch version: 2.6.0
Device count: 1


In [2]:

import os
from pathlib import Path

DATA_ROOT = Path(os.getenv("MAESTRO_ROOT", "maestro"))
MAX_FILES = int(os.getenv("MAESTRO_MAX_FILES", 24))  # Keep the training set manageable.
SAMPLE_RATE = 100  # Time steps per second for piano-roll sampling.
SEQ_LEN = 200  # Number of frames used as model context (~2 seconds at 100 fps).
LOWEST_MIDI = 21
N_PITCHES = 88
BATCH_SIZE = 8
EPOCHS = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using dataset root: {DATA_ROOT}")
print(f"Accelerator: {DEVICE}")


Using dataset root: maestro
Accelerator: cuda



> **Dataset download**: Download and extract the MAESTRO v3.0.0 MIDI archive from Google Cloud Storage:
>
> ```bash
> wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
> unzip maestro-v3.0.0-midi.zip -d maestro
> ```
>
> The extracted folder should include subdirectories such as `2004`, `2006`, ..., `2018` along with a `maestro-v3.0.0.csv` metadata file.



## 3. Data pipeline

The MAESTRO dataset contains expressive piano performances. We convert each MIDI file into a piano-roll representation at 100 frames per second, covering the 88 keys of the piano (A0 to C8). Each frame stores whether a given key is active (`1.0`) or not (`0.0`).

To keep preprocessing light, we load a configurable number of files and sample a fixed number of segments from each file for training. The result is a PyTorch `Dataset` that returns `(input, target)` pairs, where the target is the input shifted by one frame.


In [3]:
# Check that the dataset is downloaded and extracted.
if not DATA_ROOT.exists():
    raise FileNotFoundError(f"Dataset root {DATA_ROOT} does not exist. Please download and extract the MAESTRO dataset.")

# Check that the metadata file exists.
if not (DATA_ROOT / "maestro-v3.0.0.csv").exists():
    raise FileNotFoundError(f"Metadata file {DATA_ROOT / 'maestro-v3.0.0.csv'} does not exist. Please download and extract the MAESTRO dataset.")

# Check CUDA availability
if DEVICE == "cuda":
    print(f"CUDA available: {torch.cuda.is_available()}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"Device count: {torch.cuda.device_count()}")
    

CUDA available: True
CUDA version: 12.6
PyTorch version: 2.6.0
Device count: 1


In [4]:

import math
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm  # Use regular tqdm for compatibility

# NumPy compatibility patch for pretty_midi
if not hasattr(np, 'int'):
    np.int = np.int64

import pretty_midi

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)




def list_maestro_files(root: Path, max_files: int = None) -> list[Path]:
    midi_paths = sorted(root.glob("**/*.mid")) + sorted(root.glob("**/*.midi"))
    if not midi_paths:
        raise FileNotFoundError(
            f"No MIDI files were found in {root}. Check that the dataset is downloaded and extracted."
        )
    if max_files is not None:
        midi_paths = midi_paths[:max_files]
    return midi_paths


def midi_to_roll(
    path: Path,
    fs: int = SAMPLE_RATE,
    lowest: int = LOWEST_MIDI,
    n_pitches: int = N_PITCHES,
) -> np.ndarray:
    try:
        midi = pretty_midi.PrettyMIDI(str(path))
    except Exception as exc:  # pragma: no cover - defensive against malformed files
        print(f"Skipping {path}: {exc}")
        return None

    piano_roll = None
    for inst in midi.instruments:
        if inst.is_drum:
            continue
        roll = inst.get_piano_roll(fs=fs)
        if roll.size == 0:
            continue
        roll = roll[lowest : lowest + n_pitches]
        roll = (roll > 0).astype(np.float32)
        roll = roll.T  # Time first
        if piano_roll is None:
            piano_roll = roll
        else:
            # Combine instruments by taking the maximum activation
            if roll.shape[0] < piano_roll.shape[0]:
                pad = np.zeros((piano_roll.shape[0] - roll.shape[0], roll.shape[1]), dtype=np.float32)
                roll = np.vstack([roll, pad])
            elif piano_roll.shape[0] < roll.shape[0]:
                pad = np.zeros((roll.shape[0] - piano_roll.shape[0], piano_roll.shape[1]), dtype=np.float32)
                piano_roll = np.vstack([piano_roll, pad])
            piano_roll = np.maximum(piano_roll, roll)

    if piano_roll is None:
        return None

    # Trim leading/trailing silence
    active = np.where(piano_roll.sum(axis=1) > 0)[0]
    if active.size == 0:
        return None
    start, end = active[0], active[-1] + 1
    piano_roll = piano_roll[start:end]
    return piano_roll.astype(np.float32)


class MaestroRollDataset(Dataset):
    def __init__(
        self,
        midi_paths: list[Path],
        fs: int = SAMPLE_RATE,
        seq_len: int = SEQ_LEN,
        max_segments_per_file: int = 32,
    ) -> None:
        self.seq_len = seq_len
        self.rolls: list[np.ndarray] = []
        self.index: list[tuple[int, int]] = []

        for path in tqdm(midi_paths, desc="Preparing piano rolls"):
            roll = midi_to_roll(path, fs=fs)
            if roll is None or roll.shape[0] <= seq_len + 1:
                continue
            self.rolls.append(roll)
            total = roll.shape[0] - (seq_len + 1)
            segments = min(max_segments_per_file, total)
            for _ in range(segments):
                start = random.randint(0, total)
                self.index.append((len(self.rolls) - 1, start))

        if not self.index:
            raise RuntimeError("Dataset preparation failed: no usable segments were extracted.")

    def __len__(self) -> int:
        return len(self.index)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        file_idx, start = self.index[idx]
        roll = self.rolls[file_idx]
        window = roll[start : start + self.seq_len + 1]
        x = torch.from_numpy(window[:-1])
        y = torch.from_numpy(window[1:])
        return x, y


midi_files = list_maestro_files(DATA_ROOT, MAX_FILES)
dataset = MaestroRollDataset(midi_files)

# Train/validation split
val_fraction = 0.1
val_size = max(1, int(len(dataset) * val_fraction))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

len(dataset), len(train_dataloader), len(val_dataloader)


Preparing piano rolls:   0%|          | 0/24 [00:00<?, ?it/s]

Preparing piano rolls: 100%|██████████| 24/24 [00:03<00:00,  7.72it/s]


(768, 86, 10)


## 4. Model

The model is a lightweight Transformer encoder. Each frame (an 88-dimensional binary vector) is projected into the model dimension, combined with sinusoidal positional encodings, and processed by a stack of Transformer encoder layers. The output layer projects back into the piano-roll space. We use a sigmoid activation combined with a binary cross-entropy loss to model active notes independently for each pitch and frame.


In [5]:

import torch.nn as nn


class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 2048):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.pe[:, : x.size(1)]
        return self.dropout(x)


class MaestroTransformer(nn.Module):
    def __init__(
        self,
        input_dim: int = N_PITCHES,
        d_model: int = 64,
        nhead: int = 4,
        num_layers: int = 1,
        dim_feedforward: int = 512,
        dropout: float = 0.1,
    ) -> None:
        super().__init__()
        self.input = nn.Linear(input_dim, d_model)
        self.positional = PositionalEncoding(d_model, dropout)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True,
            activation="gelu",
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output = nn.Linear(d_model, input_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.input(x)
        x = self.positional(x)
        x = self.encoder(x)
        return self.output(x)


model = MaestroTransformer().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()
model


MaestroTransformer(
  (input): Linear(in_features=88, out_features=64, bias=True)
  (positional): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (output): Linear(in_features=64, out_features=88, bias=True)
)


## 5. Training

The training loop optimizes the binary cross-entropy loss over the piano-roll predictions. Training logs provide per-epoch loss averages.


In [6]:

from statistics import mean


def train_epoch(model: nn.Module, loader: DataLoader) -> float:
    model.train()
    losses = []
    for xb, yb in loader:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        losses.append(loss.item())
    return mean(losses)


def evaluate(model: nn.Module, loader: DataLoader) -> float:
    model.eval()
    losses = []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            logits = model(xb)
            loss = criterion(logits, yb)
            losses.append(loss.item())
    return mean(losses)


# Train the model and save the best model
best_model = None
best_loss = float('inf')

for epoch in range(1, EPOCHS + 1):
    train_loss = train_epoch(model, train_dataloader)
    val_loss = evaluate(model, val_dataloader)
    print(f"Epoch {epoch:02d} | train loss: {train_loss:.4f} | val loss: {val_loss:.4f}")

    if val_loss < best_loss:
        best_loss = val_loss
        best_model = model.state_dict()

torch.save(best_model, 'best_model.pth')



Epoch 01 | train loss: 0.5932 | val loss: 0.4615
Epoch 02 | train loss: 0.4194 | val loss: 0.3444
Epoch 03 | train loss: 0.3355 | val loss: 0.2804
Epoch 04 | train loss: 0.2906 | val loss: 0.2453
Epoch 05 | train loss: 0.2654 | val loss: 0.2258



## 6. Autoregressive continuation

We generate new music by seeding the model with two seconds of recorded piano-roll activations (200 frames at 100 fps) and iteratively predicting one frame at a time. The generated piano roll is converted back into a MIDI file for inspection.


In [7]:

from datetime import datetime


def sample_sequence(
    model: nn.Module,
    seed: torch.Tensor,
    steps: int,
    temperature: float = 1.0,
    threshold: float = 0.5,
    device: str = DEVICE,
) -> torch.Tensor:
    model.eval()
    device_obj = torch.device(device)
    context = seed.to(device_obj)
    generated = []
    for _ in range(steps):
        with torch.no_grad():
            logits = model(context.unsqueeze(0))[:, -1]
            if temperature != 1.0:
                logits = logits / temperature
            probs = torch.sigmoid(logits)
            next_frame = (probs > threshold).float()
        generated.append(next_frame)
        context = torch.cat([context, next_frame], dim=0)
        if context.shape[0] > SEQ_LEN:
            context = context[-SEQ_LEN:]
    return torch.cat(generated, dim=0)


def roll_to_pretty_midi(roll: np.ndarray, fs: int = SAMPLE_RATE) -> pretty_midi.PrettyMIDI:
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)
    padded_roll = np.pad(roll, ((0, 1), (0, 0)), mode="constant")
    changes = np.diff(padded_roll, axis=0)
    for pitch in range(roll.shape[1]):
        on_indices = np.where(changes[:, pitch] == 1)[0]
        off_indices = np.where(changes[:, pitch] == -1)[0]
        # Handle notes sustaining from start or to end
        if off_indices.size == 0 and on_indices.size == 0:
            continue
        if on_indices.size == 0 and off_indices.size > 0:
            on_indices = np.array([0], dtype=int)
        if off_indices.size == 0 and on_indices.size > 0:
            off_indices = np.array([roll.shape[0]], dtype=int)
        # Ensure matching pairs by clipping to min length
        pair_count = min(len(on_indices), len(off_indices))
        for start, end in zip(on_indices[:pair_count], off_indices[:pair_count]):
            if end <= start:
                continue
            note = pretty_midi.Note(
                velocity=80,
                pitch=pitch + LOWEST_MIDI,
                start=float(start) / fs,
                end=float(end) / fs,
            )
            instrument.notes.append(note)
    midi.instruments.append(instrument)
    return midi


# Load trained model or train a new one
model_path = Path("best_model.pth")
if model_path.exists():
    print("🔄 Loading trained model...")
    model = MaestroTransformer()
    model.load_state_dict(torch.load(str(model_path), map_location=DEVICE))
    model.to(DEVICE)
    model.eval()
    print("✅ Model loaded successfully!")
else:
    print("⚠️  No trained model found. Please run the training cell first (Cell 13).")
    print("🔄 Training a quick model...")
    # Use a smaller model for quick training
    model = MaestroTransformer(d_model=64, nhead=2, num_layers=1)
    model.to(DEVICE)

    # Quick training setup
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.BCEWithLogitsLoss()

    # Train for just 2 epochs on a small subset
    quick_epochs = 2
    for epoch in range(1, quick_epochs + 1):
        train_loss = train_epoch(model, train_dataloader, optimizer, criterion)
        val_loss = evaluate(model, val_dataloader, criterion)
        print(f"Quick Epoch {epoch:02d} | train loss: {train_loss:.4f} | val loss: {val_loss:.4f}")

    # Save the model
    torch.save(model.state_dict(), str(model_path))
    print("💾 Quick model saved!")

# Choose a random seed segment from the dataset
seed_x, _ = dataset[random.randrange(len(dataset))]
seed_frames = int(2 * SAMPLE_RATE)
seed = seed_x[:seed_frames]

# Generate 10 seconds of continuation (~1000 frames)
continuation = sample_sequence(model, seed, steps=1000)
continuation_roll = continuation.detach().cpu().numpy()

# Combine seed + continuation (total 12 seconds: 2s input + 10s generated)
total_roll = np.vstack([seed.numpy(), continuation_roll])

midi_out = roll_to_pretty_midi(total_roll)
out_path = Path("generated_" + datetime.now().strftime("%Y%m%d_%H%M%S") + ".mid")
midi_out.write(str(out_path))
print(f"Generated 12-second composition saved to {out_path}")
print(f"Composition: 2s seed + 10s generated = {total_roll.shape[0]/SAMPLE_RATE:.1f}s total")

# Play the generated audio
import IPython.display as ipd
try:
    # Convert MIDI to audio and play
    audio_data = midi_out.fluidsynth()
    print("🎵 Playing generated music...")
    ipd.display(ipd.Audio(audio_data, rate=44100))
    print("✅ Audio playback completed!")
except Exception as e:
    print(f"⚠️  Could not play audio: {e}")
    print("💡 You can listen to the generated MIDI file using any MIDI player")


🔄 Loading trained model...
✅ Model loaded successfully!
Generated 12-second composition saved to generated_20250923_114610.mid
Composition: 2s seed + 10s generated = 12.0s total
⚠️  Could not play audio: fluidsynth() was called but pyfluidsynth is not installed.
💡 You can listen to the generated MIDI file using any MIDI player


# 7 Create a MIDI player

In [ ]:
# Create a MIDI player
